In [ ]:
import random
min_active=8 #other 8 users recommending new events 
rec=100  #input user top 100 history 
recommentation_number = 10 # num of recommendations to output

#main function
def system(input_user_id,df_new_events, df_old_events,df_rating,min_active_users=min_active,recent=rec , rec_num = recommentation_number):
    
    
    userRated = df_rating.loc[df_rating['userId']==input_user_id]
    # new user
    if len(userRated) < 20:
        if len(df_new_events)<=rec_num:
            rec_num = len(df_new_events) -1 
        li = random.sample(range(0, len(df_new_events)-1), rec_num)
        eid =np.array(list(df_new_events['eventId']))
        
        #return random events from new events
        return list(eid[li])
        
    
    
    app_active_users = []
    
    for n_e in range(len(df_new_events)):
        if df_new_events.iloc[n_e]['eventId'] in df_rating['eventId']:
            app_active_users.extend( list(df_rating[df_rating['eventId'] == df_new_events.iloc[n_e]['eventId'] ]['userId'])  )
    app_active_users=np.unique(np.array(app_active_users))
    
    
    #Post cold start        
    similar_users_id=[]
    if len(app_active_users)> min_active_users:
        app_active_users=np.append(app_active_users,input_user_id)

        df_subset_ratings = df_rating[df_rating.userId.isin(app_active_users)]

        similar_users_id = get_similar_users(input_user_id,df_subset_ratings, df_old_events)
        similar_users_id.append(input_user_id)
        
        #get older user data.
        df_user_rating = df_rating[df_rating['userId']==input_user_id]

        recentRatings=df_user_rating.sort_values('timestamp')['eventId'].unique()[:recent]
        df_ratings_old=df_rating.loc[df_rating['eventId'].isin(recentRatings)]

        #similar users
        df_ratings_old=df_ratings_old.loc[df_ratings_old['userId'].isin(similar_users_id)]
        df_ratings_new=df_rating.loc[df_rating['eventId'].isin(list(df_new_events['eventId']))]

        #get Recommended Events...
        df_ratings_new=df_ratings_new.loc[df_ratings_new['userId'].isin(similar_users_id)]
        df_ratings_new
        
        df_all=df_ratings_new.append(df_ratings_old,ignore_index=True)
        df_pivot=df_all.pivot(index='userId',columns='eventId',values='rating').fillna(0)
        df_pivot

        R = df_pivot.to_numpy()
        mean_ =(np.mean(R, axis = 1)).reshape(-1, 1)

        U, sigma, E = svds(R - mean_, k = 5)
        sigma = np.diag(sigma)

        dot_prod =np.dot(np.dot(U, sigma),E)
        ratings_pred = dot_prod  + mean_

        preds_matrix = pd.DataFrame(ratings_pred, columns = df_pivot.columns,index=df_pivot.index)

        # return predictions
        preds_matrix = preds_matrix[list(df_ratings_new['eventId'])]

        # user_predictions = preds_matrix.loc[userID]
        events_pred=pd.unique(preds_matrix.loc[input_user_id].sort_values(ascending=False).index.to_list())[:rec_num]

        return events_pred
    
    else:
        #Check inputs again to be sure they contain the user
        userRated=df_rating.loc[df_rating['userId']==input_user_id]
        df_old_events=df_old_events.loc[df_old_events['eventId'].isin(list(userRated['eventId']))]
        
        #new(without user rating) + old events(with user ratings)
        result=df_old_events.append(df_new_events,ignore_index=True) #get old to new.

        description = result['Description'] # Description of all
        tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 3),min_df=0, stop_words='english')
        tfidf = tf.fit_transform(description)

        cosine_similarity = linear_kernel(tfidf, tfidf)

        df_cosine_sim=pd.DataFrame(data=cosine_similarity,columns=result['eventId'],index=result['eventId'])

        # old ratings -> recent=100 by default and min rating>=4
        
        min_rating=3
        
        recent_events=df_old_events.merge(df_rating,on='eventId')
        recent_events=recent_events.loc[recent_events['rating']>=min_rating].sort_values(by='timestamp',ascending=False)['eventId'][:recent]
        
        #get df slice with predicted values
        pred=df_cosine_sim.loc[df_cosine_sim.index.isin(recent_events),list(df_new_events['eventId'])]
        
        #get top 10 predictions based on max! 0th index=>Most recommended =>9th index=>10th place on recommendation
        predictions=pred.max().sort_values(ascending=False).index[:rec_num].to_list()
#         print(predictions)
        
        return predictions
        